In [1]:
import mace
import glob
import os
import torch
import warnings
warnings.filterwarnings("ignore")
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print('Using GPU')

Using GPU


In [3]:
%%writefile mace-suite/runs/oxirane/isolated/config.yml
model: 'MACE'
foundation_model: 'mace-suite/foundation_model/MACE-OFF23_medium.model'
multiheads_finetuning: True
name: 'oxirane_isolated'
model_dir: 'mace-suite/runs/oxirane/isolated/models'
log_dir: 'mace-suite/runs/oxirane/isolated/logs'
results_dir: 'mace-suite/runs/oxirane/isolated/results'
device: cuda
heads:
  spin_flip_es1:
    train_file: 'mace-suite/data/processed_datasets/oxirane/isolated/train_oxirane_sf-es1.xyz'
    valid_file : 'mace-suite/data/processed_datasets/oxirane/isolated/validation_oxirane_sf-es1.xyz'
    energy_key: 'REF_energy'
    forces_key: 'REF_forces'
    E0s: 'isolated'
  spin_flip_es2:
    train_file: 'mace-suite/data/processed_datasets/oxirane/isolated/train_oxirane_sf-es2.xyz'
    valid_file : 'mace-suite/data/processed_datasets/oxirane/isolated/validation_oxirane_sf-es2.xyz'
    energy_key: 'REF_energy'
    forces_key: 'REF_forces'
    E0s: 'isolated'

pt_train_file: 'mace-suite/data/pretrained_datasets/cleaned/cleaned_pt_train.xyz'
max_num_epochs: 700
start_swa: 475
batch_size: 5
seed: 42


Overwriting mace-suite/runs/oxirane/isolated/config.yml


In [4]:
def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]
    mace_run_train_main()

In [5]:
train_mace('mace-suite/runs/oxirane/isolated/config.yml')

2025-08-10 11:15:28.650 INFO: ===========VERIFYING SETTINGS===========
2025-08-10 11:15:28.651 INFO: Stage Two is activated as start_stage_two was defined
2025-08-10 11:15:28.651 INFO: MACE version: 0.3.14
2025-08-10 11:15:28.654 INFO: CUDA version: 12.8, CUDA device: 0
2025-08-10 11:15:29.031 INFO: Using foundation model mace-suite/foundation_model/MACE-OFF23_medium.model as initial checkpoint.
2025-08-10 11:15:29.050 INFO: Multihead finetuning mode, setting learning rate to 0.0001 and EMA to True. To use a different learning rate, set --force_mh_ft_lr=True.
2025-08-10 11:15:29.052 INFO: Using multiheads finetuning mode, setting learning rate to 0.0001 and EMA to True
2025-08-10 11:15:29.052 INFO: ===========LOADING INPUT DATA===========
2025-08-10 11:15:29.052 INFO: Using heads: ['spin_flip_es1', 'spin_flip_es2', 'pt_head']
2025-08-10 11:15:29.052 INFO: Using the key specifications to parse data:
2025-08-10 11:15:29.052 INFO: spin_flip_es1: KeySpecification(info_keys={'energy': 'REF_

In [6]:
for file in glob.glob("mace-suite/runs/oxirane/isolated/models/*.pt"):
    os.remove(file)